<a href="https://colab.research.google.com/github/GuyFarb/Cars-Project/blob/main/Copy_of_CNN6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# חיבור ל-Drive
from google.colab import drive
drive.mount('/content/drive')

# ייבוא ספריות
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# פונקציית פירוק לדוגמה מ-TFRecord
def parse_example(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(parsed['image'], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    label = parsed['label']
    return image, label

# --- טעינת סט האימון ---
train_path = "/content/drive/MyDrive/Colab Notebooks/cars196_train.tfrecord"
train_dataset = tf.data.TFRecordDataset(train_path).map(parse_example)

images_train, labels_train = [], []
for img, lbl in train_dataset:
    images_train.append(img.numpy())
    labels_train.append(lbl.numpy())

images_train = np.array(images_train)
labels_train = np.array(labels_train)
labels_train_cat = to_categorical(labels_train, num_classes=196)

print(f"✅ Train Images: {images_train.shape}, Labels: {labels_train.shape}")

# --- מודל CNN משופר ---
model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(256, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(196, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

model.summary()

# --- Data Augmentation ---
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

# --- אימון (10 אפוקים) ---
history = model.fit(
    datagen.flow(images_train, labels_train_cat, subset='training', batch_size=64),
    validation_data=datagen.flow(images_train, labels_train_cat, subset='validation', batch_size=64),
    epochs=10,
    callbacks=callbacks
)

# --- שמירת המודל לאחר אימון
model.save("cnn_model_after_train.h5")
print("✅ המודל לאחר אימון נשמר בשם cnn_model_after_train.h5")

# ===============================================
# 🔹 שלב ה-Test
# ===============================================
test_path = "/content/drive/MyDrive/Colab Notebooks/cars196_test.tfrecord"
test_dataset = tf.data.TFRecordDataset(test_path).map(parse_example)

images_test, labels_test = [], []
for img, lbl in test_dataset:
    images_test.append(img.numpy())
    labels_test.append(lbl.numpy())

images_test = np.array(images_test)
labels_test = np.array(labels_test)

# --- חיזוי על סט הבדיקה
predictions_test = model.predict(images_test, batch_size=64)
y_pred = np.argmax(predictions_test, axis=1)
y_true = labels_test

# --- דיוק ודו"ח
report = classification_report(y_true, y_pred, digits=4, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv("classification_report_test.csv")
print(report_df.head(50))

# --- דיוק כולל
acc = accuracy_score(y_true, y_pred)
print(f"🎯 דיוק סופי על סט ה-Test: {acc:.4f}")

# --- שמירת המודל לאחר Test
model.save("cnn_model_after_test.h5")
print("✅ המודל לאחר Test נשמר בשם cnn_model_after_test.h5")

# --- גרף ביצועים
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('📊 Accuracy לאורך Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('📉 Loss לאורך Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# --- Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(14,12))
sns.heatmap(cm, cmap="Blues", square=False, xticklabels=False, yticklabels=False)
plt.title("🌀 Confusion Matrix על סט ה-Test")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


Mounted at /content/drive
✅ Train Images: (8144, 224, 224, 3), Labels: (8144,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 224, 224, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 112, 112, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 56, 56, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 28, 28, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 256)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 196)                 │         100,548 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 622,468 (2.37 MB)

 Trainable params: 621,508 (2.37 MB)

 Non-trainable params: 960 (3.75 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 795s 8s/step - accuracy: 0.0092 - loss: 5.3603 - top_k_categorical_accuracy: 0.0371 - val_accuracy: 0.0037 - val_loss: 5.6647 - val_top_k_categorical_accuracy: 0.0240 - learning_rate: 0.0010
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 754s 7s/step - accuracy: 0.0171 - loss: 5.1411 - top_k_categorical_accuracy: 0.0673 - val_accuracy: 0.0068 - val_loss: 6.6158 - val_top_k_categorical_accuracy: 0.0246 - learning_rate: 0.0010
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.0304 - loss: 5.0193 - top_k_categorical_accuracy: 0.0925
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
102/102 ━━━━━━━━━━━━━━━━━━━━ 802s 7s/step - accuracy: 0.0303 - loss: 5.0194 - top_k_categorical_accuracy: 0.0925 - val_accuracy: 0.0061 - val_loss: 6.8546 - val_top_k_categorical_accuracy: 0.0240 - learning_rate: 0.0010
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 751s 7s/step - accuracy: 0.0309 - loss: 4.8802 - top_k_categorical_accuracy: 0.1

✅ המודל לאחר אימון נשמר בשם cnn_model_after_train.h5


In [ ]:
# 🔸 חיבור ל-Drive
from google.colab import drive
drive.mount('/content/drive')

# 🔸 ייבוא ספריות
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 🔸 פונקציית פירוק לדוגמה מ-TFRecord
def parse_example(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(parsed['image'], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    label = parsed['label']
    return image, label

# 🔸 קריאה מהקובץ של ה-Train
train_path = "/content/drive/MyDrive/Colab Notebooks/cars196_train.tfrecord"
train_dataset = tf.data.TFRecordDataset(train_path).map(parse_example)

# המרה ל-NumPy
images_train, labels_train = [], []
for img, lbl in train_dataset:
    images_train.append(img.numpy())
    labels_train.append(lbl.numpy())

images_train = np.array(images_train)
labels_train = np.array(labels_train)
labels_train_cat = to_categorical(labels_train, num_classes=196)

print(f"✅ Train Images: {images_train.shape}, Labels: {labels_train.shape}")

# 🔸 בניית מודל CNN משופר
model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(256, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(196, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

model.summary()

# 🔸 Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

# 🔸 Callbacks עם סבלנות גבוהה
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=10, mode='max', restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, mode='max')
]

# 🔸 אימון
history = model.fit(
    datagen.flow(images_train, labels_train_cat, subset='training', batch_size=64),
    validation_data=datagen.flow(images_train, labels_train_cat, subset='validation', batch_size=64),
    epochs=10,
    callbacks=callbacks
)

# 🔸 שמירת המודל לאחר אימון
model.save("cnn_model_after_train.h5")
print("✅ המודל לאחר אימון נשמר בשם cnn_model_after_train.h5")

# ===============================================
# 🔸 🔹 🔸 שלב ה-Test 🔸 🔹 🔸
# ===============================================
test_path = "/content/drive/MyDrive/Colab Notebooks/cars196_test.tfrecord"
test_dataset = tf.data.TFRecordDataset(test_path).map(parse_example)

images_test, labels_test = [], []
for img, lbl in test_dataset:
    images_test.append(img.numpy())
    labels_test.append(lbl.numpy())

images_test = np.array(images_test)
labels_test = np.array(labels_test)

# חיזוי על סט הבדיקה
predictions_test = model.predict(images_test, batch_size=64)
y_pred = np.argmax(predictions_test, axis=1)
y_true = labels_test

# --- דיוק ודו"ח
report = classification_report(y_true, y_pred, digits=4, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv("classification_report_test.csv")
print(report_df.head(50))

acc = accuracy_score(y_true, y_pred)
print(f"\n🎯 דיוק סופי על סט ה-Test: {acc:.4f}")

# 🔸 שמירת המודל לאחר Test
model.save("cnn_model_after_test.h5")
print("✅ המודל לאחר Test נשמר בשם cnn_model_after_test.h5")

# 🔸 גרף דיוק והפסד
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('📊 Accuracy לאורך Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('📉 Loss לאורך Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 🔸 Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(14,12))
sns.heatmap(cm, cmap="Blues", square=False, xticklabels=False, yticklabels=False)
plt.title("🌀 Confusion Matrix על סט ה-Test")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Train Images: (8144, 224, 224, 3), Labels: (8144,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 224, 224, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 112, 112, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 56, 56, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 28, 28, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 256)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 196)                 │         100,548 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 622,468 (2.37 MB)

 Trainable params: 621,508 (2.37 MB)

 Non-trainable params: 960 (3.75 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 860s 8s/step - accuracy: 0.0059 - loss: 5.3687 - top_k_categorical_accuracy: 0.0331 - val_accuracy: 0.0068 - val_loss: 5.7633 - val_top_k_categorical_accuracy: 0.0233 - learning_rate: 0.0010
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 853s 8s/step - accuracy: 0.0206 - loss: 5.1292 - top_k_categorical_accuracy: 0.0735 - val_accuracy: 0.0049 - val_loss: 6.1655 - val_top_k_categorical_accuracy: 0.0240 - learning_rate: 0.0010
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 867s 9s/step - accuracy: 0.0278 - loss: 4.9830 - top_k_categorical_accuracy: 0.0991 - val_accuracy: 0.0043 - val_loss: 6.3940 - val_top_k_categorical_accuracy: 0.0332 - learning_rate: 0.0010
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 861s 8s/step - accuracy: 0.0397 - loss: 4.9041 - top_k_categorical_accuracy: 0.1188 - val_accuracy: 0.0104 - val_loss: 5.7359 - val_top_k_categorical_accuracy: 0.0528 - learning_rate: 0.0010
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 853s 8s/step - accuracy: 0.0424 - lo

✅ המודל לאחר אימון נשמר בשם cnn_model_after_train.h5


In [ ]:
# חיבור לגוגל דרייב
from google.colab import drive
drive.mount('/content/drive')

# ייבוא ספריות
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# פונקציית פירוק TFRecord
def parse_example(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(parsed['image'], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    label = parsed['label']
    return image, label

# טעינת סט האימון
train_path = "/content/drive/MyDrive/Colab Notebooks/cars196_train.tfrecord"
train_dataset = tf.data.TFRecordDataset(train_path).map(parse_example)

images_train, labels_train = [], []
for img, lbl in train_dataset:
    images_train.append(img.numpy())
    labels_train.append(lbl.numpy())

images_train = np.array(images_train)
labels_train = np.array(labels_train)
labels_train_cat = to_categorical(labels_train, num_classes=196)

print(f"✅ Train Images: {images_train.shape}, Labels: {labels_train.shape}")

# 🔸 CNN משופר
model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(256, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(512, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(196, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

model.summary()

# 🔸 Data Augmentation משופר
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=6, restore_best_weights=True, mode='max', verbose=1),
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, mode='max', verbose=1)
]

# 🔸 אימון המודל (Train)
history = model.fit(
    datagen.flow(images_train, labels_train_cat, subset='training', batch_size=64),
    validation_data=datagen.flow(images_train, labels_train_cat, subset='validation', batch_size=64),
    epochs=20,
    callbacks=callbacks
)

# 🔸 שמירה לאחר אימון
model.save("cnn_model_improved_after_train.h5")

# ===============================================
# 🔸 🔹 🔸 שלב ה-Test 🔸 🔹 🔸
# ===============================================
test_path = "/content/drive/MyDrive/Colab Notebooks/cars196_test.tfrecord"
test_dataset = tf.data.TFRecordDataset(test_path).map(parse_example)

images_test, labels_test = [], []
for img, lbl in test_dataset:
    images_test.append(img.numpy())
    labels_test.append(lbl.numpy())

images_test = np.array(images_test)
labels_test = np.array(labels_test)

# תחזית
predictions_test = model.predict(images_test, batch_size=64)
y_pred = np.argmax(predictions_test, axis=1)
y_true = labels_test

# דוח ביצועים
report = classification_report(y_true, y_pred, digits=4, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv("cnn_improved_classification_report_test.csv")
print(report_df.head(50))

# דיוק סופי
acc = accuracy_score(y_true, y_pred)
print(f"\n🎯 דיוק סופי על סט ה-Test: {acc:.4f}")

# שמירת המודל לאחר Test
model.save("cnn_model_improved_after_test.h5")

# גרפים
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('📊 Accuracy לאורך Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('📉 Loss לאורך Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(14,12))
sns.heatmap(cm, cmap="Blues", square=False, xticklabels=False, yticklabels=False)
plt.title("🌀 Confusion Matrix על סט ה-Test")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Train Images: (8144, 224, 224, 3), Labels: (8144,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 224, 224, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 112, 112, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 56, 56, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 28, 28, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 14, 14, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 14, 14, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 1,935,748 (7.38 MB)

 Trainable params: 1,933,764 (7.38 MB)

 Non-trainable params: 1,984 (7.75 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 928s 9s/step - accuracy: 0.0102 - loss: 5.4658 - top_k_categorical_accuracy: 0.0348 - val_accuracy: 0.0068 - val_loss: 5.8284 - val_top_k_categorical_accuracy: 0.0313 - learning_rate: 0.0010
Epoch 2/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 911s 9s/step - accuracy: 0.0153 - loss: 5.1941 - top_k_categorical_accuracy: 0.0600 - val_accuracy: 0.0037 - val_loss: 7.3048 - val_top_k_categorical_accuracy: 0.0246 - learning_rate: 0.0010
Epoch 3/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 910s 9s/step - accuracy: 0.0168 - loss: 5.1405 - top_k_categorical_accuracy: 0.0716 - val_accuracy: 0.0049 - val_loss: 6.9279 - val_top_k_categorical_accuracy: 0.0221 - learning_rate: 0.0010
Epoch 4/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 934s 9s/step - accuracy: 0.0173 - loss: 5.0884 - top_k_categorical_accuracy: 0.0743 - val_accuracy: 0.0111 - val_loss: 6.1656 - val_top_k_categorical_accuracy: 0.0338 - learning_rate: 0.0010
Epoch 5/20
102/102 ━━━━━━━━━━━━━━━━━━━━ 935s 9s/step - accuracy: 0.0317 - lo

KeyboardInterrupt: 